In [1]:

import requests 
from bs4 import BeautifulSoup 
  
def news(url): 
    list_ = []
    # the target we want to open     
    
      
    #open with GET method 
    resp=requests.get(url) 
      
    #http_respone 200 means OK status 
    if resp.status_code==200: 
      
        # we need a parser,Python built-in HTML parser is enough . 
        soup=BeautifulSoup(resp.text,'html.parser')     
  
        # l is the list which contains all the text i.e news  
    
        href =soup.findAll("h2",{"class":"c_title"}) 
        for l in href:
            l = str(l)

            st = l[l.find('href')+4:]
            st = st[:st.find('>')]
            id_= ''
            x = st.split('/')
            id_ = x[len(x)-1]
            #print(id_[:id_.find('"')])


            list_.append(id_[:id_.find('"')])
        #now we want to print only the text part of the anchor. 
        #find all the elements of a, i.e anchor 
        #for i in l.findAll("a"): 
        #    print(i.text) 
    else: 
        print("Error")
    return list_




In [11]:
import numpy as np
categories = ['animals','anime','art-design','architecture','auto-moto','celebrities','education','travel',
           'fashion-beauty','food','feetness','gaming','health','magezine','music','nature','news','photo','sport',
             'science','technology','video','other','economic','religion','cooking','Romantic','movie','shop']
my_ids = []
j = 1
channels ={}
channels['iribnews'] = [-1,0]
for i in categories:
    url='https://mychannels.ir/soroush/'+i
    ids = news(url)
    print(ids)
    for d in ids:
        channels[d] = [-1,j]
        j+=1
        my_ids.append(d)
    

['animalshelter', 'bazar_dam', 'phoenix_ilove', 'natgeo', 'siadpaliz', 'zalooparvar.fadak', 'teyur966', 'Just_Fish', 'poultry96', 'LarrySerra', 'amozeshkadecanary', 'national_geographik', 'aquarium_fish', 'akvaruom.farzad', 'toyyyour']
['crtoonland', 'otaku.net', 'behtarinanime', 'animefun', 'hamidmixer', 'koodakonline1', 'cartoonstv1', 'animation_kotah', 'andeshezeba', 'clip_irancartoon', 'og_instagram', 'clip_kodakane', 'tom_and_jery', 'clip_babsfangi', 'clip_cartoon']
['tarh.graph', 'Tablofarsh.chehre.mandegar', 'logowhiteblack', 'avindordoneh', 'fatjmazahrayas', 'artist_painter', 'Mr.geraphic', 'namadarzan', 'ms_logo.ir', 'Angoshtar_khatam', 'sobhanms', 'parvanjf', 'epoxsiii', 'naziricalligraphy', 'Kaghzqom']
['kayerpipe', '3dmax_farsi', 'cc.architect', 'mojriha', 'decokara', 'urbanity', 'MAXWORLD_IR', 'decoraat', 'ostadekargari']
['WAcars', '1dande_ir', 'Water_Pump_TakTaz', 'machine_SPORT', 'abolfazl_prop', 'kolahkasket.ir', 'carkhavaran', 'sajadhafizi', 'VisionSport', 'lux_motor'

In [12]:
# channels

In [4]:
# !python -m pip install pykafka

import hashlib
import requests
import json
from time import sleep
import sseclient
def get_channel_data(channel_name, size, last_message_id):
    endpoint = 'channel/archive'
    a = "95d58639"
    b = "24c0"
    c = "4b72"
    d = "80a5"
    f = "e124f41d7af9"
    api_secret = "-".join([a,b,c,d,f])
    api_resource = 'channel/archive'
    clinet_id = '7743461522282941752'
    format = 'json'
    api_version = 'v1'
    if last_message_id != -1:
        message = f'{{"channel_id":"{channel_name}", "limit": {size}, "message_id": "{last_message_id}"}}'
    else:
        message = f'{{"channel_id":"{channel_name}", "limit": {size}}}'
    digest = hashlib.md5((api_secret+api_version+format+clinet_id+api_resource+message).encode('utf-8')).hexdigest()
    url = f"https://what.sapp.ir/srvcs-app/v1/json/7743461522282941752/{digest}/channel/archive"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36',
        'keep_alive':'False'
    }
    

    rq = requests.post(url, data = message.encode("utf-8"), headers=headers)
    res = json.loads(rq.text)
    return res

In [14]:
import re
from pykafka import KafkaClient
client = KafkaClient(hosts="127.0.0.1:9092")
topic = client.topics["final"]

main_words = ['بورس','اقتصاد','تحریم','دولت','دلار','طلا','کرونا','شاخص بورس','تورم','دانشگاه','سقوط','رشد']
channels = {'iribnews':[-1,0]}
while(True):
    for channel,msgid in channels.items():
        out = get_channel_data(channel,2,-1)
        if type(out) == list:
#             print('Oops')
            continue
#         print(out)
        if(out['total']>0):
            if(out['messages'][0]['message_id'] != msgid[0]):
                channels[channel][0] = out['messages'][0]['message_id']
                out['messages'][0]['index'] = msgid[1]
                out['messages'][0]['metadata'] = [tag.strip("#") for tag in out['messages'][0]['text'].split() if tag.startswith("#")] + out['messages'][0]['frequency_word'] + [word for word in main_words if word in out['messages'][0]['text']]
                out['messages'][0]['hashtags'] = [tag.strip("#") for tag in out['messages'][0]['text'].split() if tag.startswith("#")]
                out['messages'][0]['links'] = re.findall(r'(https?://[^\s]+)', out['messages'][0]['text'])
                if out['messages'][0]['file']['extention']=='jpg':
                    out['messages'][0]['images'] = [out['messages'][0]['file']['url']]
                else :
                    out['messages'][0]['images'] = []
    #             print(out['messages'][0])
                print('*')
                with topic.get_sync_producer() as producer:
                    producer.produce(bytes(str(out['messages'][0]), encoding='utf-8'))
    sleep(60)


*


KeyboardInterrupt: 